In [1]:
from transformers import AutoTokenizer, RagRetriever, RagSequenceForGeneration
import torch

/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq", index_name="exact"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
/home/junseokkim00/anaconda3/envs/rp2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEnc

DatasetGenerationError: An error occurred while generating the dataset

In [ ]:
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

In [23]:
inputs = tokenizer("Which magazine was started first Arthur's Magazine or First for Women?", return_tensors="pt")

In [24]:
input_ids = inputs["input_ids"]

In [25]:
question_hidden_states = model.question_encoder(input_ids)[0]

In [26]:
docs_dict = retriever(input_ids.numpy(), question_hidden_states.detach().numpy(), return_tensors="pt")

In [27]:
doc_scores = torch.bmm(
    question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2)
).squeeze(1)

In [ ]:
tokenizer.batch_decode(docs_dict['context_input_ids'])

In [ ]:
model.